### DoRA Fine-tuning


Here, we'll be using the DoRA technique to fine-tune our model.


TODO: Work on the LEAN dataset 

In [ ]:
# Wandb login

import wandb
from dotenv import load_dotenv, find_dotenv
import os
import getpass

load_dotenv(find_dotenv())

def get_api_key(env_var, prompt):
    if not os.getenv(env_var):
        os.environ[env_var] = getpass.getpass(prompt)

get_api_key("WANDB_API_KEY", "Enter your Weights & Biases API key: ")
wandb.login(key=os.getenv("WANDB_API_KEY"))

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoTokenizer, Qwen3ForSequenceClassification, default_data_collator, EarlyStoppingCallback, TrainingArguments, Trainer
import torch
import os 
import pandas as pd
from datasets import Dataset, DatasetDict

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_GPUS = torch.cuda.device_count()
BF16 = torch.cuda.is_bf16_supported()

os.environ["HF_HOME"] =  os.path.join(os.environ["SCRATCH"], "huggingface_cache")
os.environ["HF_HUB_CACHE"]       = os.path.join(os.environ["HF_HOME"], "hub")
os.environ["TRANSFORMERS_CACHE"] = os.path.join(os.environ["HF_HOME"], "models")
os.environ["HF_DATASETS_CACHE"]  = os.path.join(os.environ["HF_HOME"], "datasets")


cache_dir = os.environ["HF_HOME"]


Loading our model

In [ ]:
model_name = "Qwen/Qwen3-1.7B"
model = Qwen3ForSequenceClassification.from_pretrained(model_name, 
                                                       dtype=torch.float32, #use bf16 if supported 
                                                       device_map="auto",
                                                       num_labels=3,
                                                       trust_remote_code=True,
                                                       cache_dir=cache_dir)
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                         cache_dir=cache_dir)


Loading the PEFT Trainer

In [ ]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=[
        "q_proj","v_proj","k_proj","o_proj",
        "gate_proj","up_proj","down_proj",
        "lora_magnitude_vector"  # if using DoRa
    ],
    lora_dropout=0.01,
    bias="none",
    task_type=TaskType.SEQ_CLS, # TODO: Change this to text generation
    use_dora=True,
)

